In [1]:
import polars as pl
from polars_order_book import calculate_bbo

In [2]:
n = 10_000_000
market_data = pl.DataFrame({
    'price': [1, 2, 3, 6, 5, 4, 3, 1, 2, 5, 4, 6] * n,
    'qty':   [1, 2, 3, 6, 5, 4, -3, -1, -2, -5, -4, -6] * n,
    'is_bid': [True, True, True, False, False, False, True, True, True, False, False, False] * n
}, schema={
    'price': pl.Int64,
    'qty': pl.Int64,
    'is_bid': pl.Boolean
})

In [3]:
%%timeit 
market_data.with_columns(
    bbo=calculate_bbo('price', 'qty', 'is_bid')
).unnest('bbo')

44.8 s ± 119 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
n = 10_000_000
market_data = pl.DataFrame({
    'price': [1, 2, 3, 6, 5, 4, 3, 1, 2, 5, 4, 6, 1, 2, 3, 6, 5, 4, 3, 1, 2, 5, 4, 6] * (n//2),
    'qty':   [1, 2, 3, 6, 5, 4, 3, 1, 2, 5, 4, 6, -1, -2, -3, -6, -5, -4, -3, -1, -2, -5, -4, -6] * (n//2),
    'is_bid': [True, True, True, False, False, False, True, True, True, False, False, False] * n
}, schema={
    'price': pl.Int64,
    'qty': pl.Int64,
    'is_bid': pl.Boolean
})

In [28]:
%timeit market_data.with_columns(bbo=calculate_bbo('price', 'qty', 'is_bid')).unnest('bbo')

3.9 s ± 32 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
n = 10_000_000
symbols = ['A', 'B', 'C', 'D', 'E']
symbols = sum([[s] * 12 for s in symbols], []) 
market_data = pl.DataFrame({
    'price': [1, 2, 3, 6, 5, 4, 3, 1, 2, 5, 4, 6] * n,
    'qty':   [1, 2, 3, 6, 5, 4, -3, -1, -2, -5, -4, -6] * n,
    'is_bid': [True, True, True, False, False, False, True, True, True, False, False, False] * n,
    'symbol': symbols * (n // 5)
}, schema={
    'price': pl.Int64,
    'qty': pl.Int64,
    'is_bid': pl.Boolean,
    'symbol': pl.Utf8
})

In [ ]:
market_data.group_by('symbol').agg(
    calculate_bbo('price', 'qty', 'is_bid').alias('bbo')
).explode('bbo').unnest('bbo')